In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
credential = DefaultAzureCredential()

In [ ]:
ml_client = MLClient(
    credential=credential,
    subscription_id="<SUBSCRIPTION_ID>",
    resource_group_name="<RESOURCE_GROUP>",
    workspace_name="<AML_WORKSPACE_NAME>",
)

In [ ]:
from azure.ai.ml.entities import AmlCompute

# コンピューティングクラスターの名前を設定
cpu_compute_target = "cpu-cluster"

try:
    # 同名のコンピューティングクラスターが作成されているか確認
    cpu_cluster = ml_client.compute.get(cpu_compute_target)
    print(
        f"{cpu_compute_target}と同一名称のクラスタが存在するので既存クラスタを使用します"
    )

except Exception:
    print("新しいコンピューティングクラスターを作成します")

    # コンピューティングクラスター作成のためのパラメータ指定
    cpu_cluster = AmlCompute(
        name=cpu_compute_target ,
        # オンデマンドタイプのVM指定
        type="amlcompute",
        # VMサイズ
        size="Standard_D2s_v3",
        # 最小ノード数
        min_instances=0,
        # 最大ノード数
        max_instances=3,
        # クラスタをスケールダウンするまでのノードのアイドル時間(秒)
        idle_time_before_scale_down=180,
        # VM優先度レベル(LowPriority(低優先度)はプリエンティブルVMですが、Dedicated(専用)よりコスト安)
        tier="Dedicated",
    )
    print(
    f"クラスタ名 {cpu_cluster.name} 、VMサイズ{cpu_cluster.size}でクラスタ作成します"
    )
    # クラスタ作成開始
    cpu_cluster = ml_client.compute.begin_create_or_update(cpu_cluster)


In [ ]:
import os

dependencies_dir = "./dependencies"
os.makedirs(dependencies_dir , exist_ok=True)

In [ ]:
%%writefile {dependencies_dir}/conda.yml
name: model-env
channels:
    - conda-forge
dependencies:
    - python=3.8
    - numpy=1.21.2
    - pip=21.2.4
    - scikit-learn=0.24.2
    - scipy=1.7.1
    - pandas>=1.1,<1.2
    - pip:
        - inference-schema[numpy-support]==1.3.0
        - xlrd==2.0.1
        - mlflow== 1.26.1
        - azureml-mlflow==1.42.0
        - psutil>=5.8,<5.9
        - tqdm>=4.59,<4.60
        - ipykernel~=6.0
        - matplotlib

In [ ]:
from azure.ai.ml.entities import Environment

custom_env_name = "aml-scikit-learn"

pipeline_job_env = Environment(
    name=custom_env_name ,
    description="クレジットカードのデフォルトを推論するモデルを学習するためのカスタム環境",
    tags={"scikit-learn": "0.24.2"},
    conda_file=os.path.join(dependencies_dir , "conda.yml"),
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest",
)
pipeline_job_env = ml_client.environments.create_or_update(pipeline_job_env)

print(
    f"{pipeline_job_env.name} 環境をAzureMLワークスペースへ登録しました。環境バージョンは {pipeline_job_env.version} です。"
)

In [ ]:
import os

train_src_dir = "./src"
os.makedirs(train_src_dir , exist_ok=True)

In [ ]:
%%writefile {train_src_dir}/main.py
import os
import argparse
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

def main():
    """メイン関数"""
    #パラメータ
    parser = argparse.ArgumentParser()

    parser.add_argument("--data", type=str, help="入力データのパス")
    parser.add_argument("--test_train_ratio", type=float , required=False , default=0.25)
    parser.add_argument("--n_estimators", required=False, default=100, type=int)
    parser.add_argument("--learning_rate", required=False, default=0.1, type=float )
    parser.add_argument("--registered_model_name", type=str, help="登録するモデル名")
    args = parser.parse_args()
    
    # ロギング開始
    mlflow.start_run()
    # 自動ロギング有効化
    mlflow.sklearn.autolog()

    ###################
    #<データ準備>
    ###################
    print("␣".join(f"{k}={v}" for k, v in vars(args).items()))
    print("入力データ:", args.data)
    credit_df=pd.read_excel(args.data, header=1, index_col=0)

    train_df, test_df=train_test_split(
        credit_df ,
        test_size=args.test_train_ratio,
    )
    ####################
    #</データ準備>
    ####################

    ##################
    #<学習>
    ##################
    # 学習データからターゲット列を取得
    y_train = train_df.pop("default payment next month")

    # array型へ変換
    X_train = train_df.values

    # テストデータからターゲット列を取得
    y_test = test_df.pop("default payment next month")

    # array型へ変換
    X_test = test_df.values
    print(f"学習データの形状{X_train.shape}")
    
    clf = GradientBoostingClassifier(
        n_estimators=args.n_estimators, learning_rate=args.learning_rate
    )
    clf.fit(X_train , y_train)

    y_pred = clf.predict(X_test)
    print(classification_report(y_test, y_pred))
    ###################
    #</学習>
    ###################

    ##########################
    #<モデル登録>
    ##########################
    # 学習済みモデルをAzureMLワークスペースへ登録
    print("MLflowでモデル登録")
    mlflow.sklearn.log_model(
        sk_model=clf,
        registered_model_name=args.registered_model_name,
        artifact_path=args.registered_model_name,
    )
    ###########################
    #</モデル登録>
    ###########################
    # ロギング停止
    mlflow.end_run()

if __name__ == "__main__":
    main()


In [ ]:
from azure.ai.ml import command
from azure.ai.ml import Input

registered_model_name = "credit_defaults_model"

job = command(
    # 学習スクリプトの引数
    inputs=dict(
        # 学習データのパス
        data=Input(
            type="uri_file",
            path="https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls",
        ),
        # テストデータの分割率
        test_train_ratio=0.2,
        # 習率
        learning_rate=0.25,
        # 登録するモデル名
        registered_model_name=registered_model_name,
    ),
    # 学習スクリプトの格納場所
    code="./src/",
    # 学習スクリプトの実行コマンド
    command="python main.py --data ${{inputs.data}} --test_train_ratio ${{inputs.test_train_ratio}} --learning_rate ${{inputs.learning_rate}} --registered_model_name ${{inputs.registered_model_name}}",
    # 環境 (@latestで最新版を指定。:バージョン数でバージョン指定も可)
    environment="aml-scikit-learn@latest",
    # 学習先のコンピューティングクラスター
    compute="cpu-cluster",
    # 実験名
    experiment_name="train_model_credit_default_prediction",
    # ジョブの表示名
    display_name="credit_default_prediction",
)

In [ ]:
job = ml_client.create_or_update(job)
# ジョブ実行が完了するまで待機
ml_client.jobs.stream(job.name)

In [ ]:
import uuid

# ユニークなエンドポイント名を作成
online_endpoint_name = "credit-endpoint-" + str(uuid.uuid4())[:8]

from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
)

# マネージドオンラインエンドポイントを作成
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="クレジットカードのデフォルト推論用のマネージドオンラインエンドポイント",
    auth_mode="key",
    tags ={
        "training_dataset": "credit_defaults",
        "model_type": "sklearn.GradientBoostingClassifier",
    },
)

endpoint = ml_client.online_endpoints.begin_create_or_update(endpoint).result()

print(f"エンドポイント名 {endpoint.name} プロビジョニングステータス: {endpoint.provisioning_state}")


In [ ]:
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)
print(
    f'ステータスが "{endpoint.provisioning_state}" のエンドポイント "{endpoint.name}" を取得'
)

In [ ]:
# モデルの最新のバージョン番号取得 
latest_model_version = max(
    [int(m.version) for m in ml_client.models.list(name=registered_model_name)]
)
print(f'Latest model is version "{latest_model_version}" ')

# デプロイするモデルを選択、登録済みモデルの最新バージョンを使用
model = ml_client.models.get(name=registered_model_name, version=latest_model_version)

# マネージドオンラインエンドポイントへのデプロイメントを作成
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_DS3_v2",
    instance_count=1,
)

# デプロイ実行
blue_deployment = ml_client.begin_create_or_update(blue_deployment).result()

In [ ]:
deploy_dir = "./deploy"
os.makedirs(deploy_dir, exist_ok=True)

In [ ]:
%%writefile {deploy_dir}/sample-request.json
{
  "input_data": {
    "columns": [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22],
    "index": [0, 1],
    "data": [
            [20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0],
            [10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 10, 9, 8]
        ]
  }
}

In [ ]:
# マネージドオンラインエンドポイントで推論実行
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    request_file="./deploy/sample-request.json",
    deployment_name="blue",
)

In [ ]:
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)

.